# Trabalho 2 de Paradigmas da Programação

## Bibliotecas

In [4]:
import pandas as pd
import sparql_dataframe

from functools import reduce

## Sparql query

In [5]:
endpoint = "http://dbpedia.org/sparql"

In [6]:
query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select distinct ?nome ?paisNascimento ?dataNascimentoFormatted ?clubeNome ?posicaoLabel ?altura ?numeroCamisa ?allGols ?liga where {
  
  # Pegando as instâncias de SoccerPlayer e o nome do jogador.
  { ?sub rdf:type dbo:SoccerPlayer . }
  union
  { ?sub a dbo:SoccerPlayer . }
  ?sub rdfs:label ?nome .
  FILTER (LANG(?nome) = 'en') .
  
  # Pegando o país de nascimento do jogador.
  ?sub dbo:birthPlace ?localNascimento .
  OPTIONAL { 
    ?localNascimento dbo:country ?paisNascimentoURI . 
  	?paisNascimentoURI rdfs:label ?paisNascimento .
   	FILTER (LANG(?paisNascimento) = 'en') .
  }
  
  # Pegando a data de nascimento do jogador.
  ?sub dbp:birthDate ?dataNascimento .
  
  # Pegando o clube em que o jogador está jogando.
  ?sub dbp:currentclub ?clubeAtual .
  ?clubeAtual rdfs:label ?clubeNome .
  FILTER (LANG(?clubeNome) = 'en') .
  
  # Pegando a posição em que o jogador joga.
  ?sub dbp:position ?posicao .
  ?posicao rdfs:label ?posicaoLabel .
  FILTER (LANG(?posicaoLabel) = 'en') .
  
  # Pegando a altura do jogador.
  ?sub dbo:height ?altura .
  FILTER (datatype(?altura) = xsd:double) .
  
  # Pegando o número da camisa do jogador.
  ?sub dbp:clubnumber ?numeroCamisa .
  FILTER (datatype(?numeroCamisa) = xsd:integer) .
  
  # Pegando a lista de gols do jogador, onde apenas o máximo dessa lista representa a quantidade real de gols dele.
  ?sub dbp:goals ?allGols .
  FILTER (datatype(?allGols) = xsd:integer) .
  
  # Pegando a liga em que o jogador joga.
  ?clubeAtual dbo:league ?ligaURI .
  ?ligaURI rdfs:label ?liga .
  FILTER (LANG(?liga) = 'en') .
  
  
  # Filtrando variáveis que devem ser URIs.
  FILTER (isURI(?clubeAtual)) .
  FILTER (isURI(?paisNascimentoURI)) .
  
  # Transformando as datas de nascimento do formato dateTime para date.
  BIND (
    IF (datatype(?dataNascimento) = xsd:dateTime, STRDT(STR(?dataNascimento), xsd:date), ?dataNascimento)
    AS ?dataNascimentoFormatted
  )
}"""

In [7]:
df = sparql_dataframe.get(endpoint, query)
df.head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
0,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,2,Calcutta Football League
1,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,7,Calcutta Football League
2,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,1,Calcutta Football League
3,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,0,Calcutta Football League
4,Wut Tola,Cambodia,2002-10-06,Visakha FC,Forward (association football),1.70,18,1,Cambodian Premier League


In [8]:
df.shape

(8639, 9)

Iremos filtrar o dataframe para tirar as repetições de jogadores

Nesta parte foi utilizada a funcionalidade **```filter```**

In [9]:
jogadores_list = df.to_dict(orient='records')

jogadores_filtrados = list(filter(lambda j: j['allGols'] == max(
    jogador['allGols'] for jogador in jogadores_list if jogador['nome'] == j['nome']), jogadores_list))

In [10]:
new_df = pd.DataFrame(jogadores_filtrados)
new_df.head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
0,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,7,Calcutta Football League
1,Wut Tola,Cambodia,2002-10-06,Visakha FC,Forward (association football),1.70,18,3,Cambodian Premier League
2,Chhom Pisa,Cambodia,1995-03-03,Phnom Penh Crown FC,Centre back,1.70,39,0,Cambodian Premier League
3,Sa Ty,Cambodia,2002-04-04,Visakha FC,Forward (association football),1.70,29,10,Cambodian Premier League
4,Hoy Phallin,Cambodia,1996-03-30,ISI Dangkor Senchey FC,Midfielder,1.70,72,2,Cambodian League 2


In [11]:
new_df.shape

(2597, 9)

## Regras

### Regra 1

Acha todos os jogadores que são conteporâneos, ou seja, nasceram no mesmo ano

Nesta regra foi utilizada a funcionalidade **```map```**

In [12]:
def jogadores_contemporaneos(df):
    return [(j1, j2) for j1, ano1 in zip(df['nome'], list(map(lambda date: int(date[:4]) if '-' in date else None, df["dataNascimentoFormatted"])))
                     for j2, ano2 in zip(df['nome'], list(map(lambda date: int(date[:4]) if '-' in date else None, df["dataNascimentoFormatted"])))
                     if j1 < j2 and ano1 == ano2]

In [13]:
jogadores_contemporaneos(new_df)

[('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Zé Roberto (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Zé Rafael'),
 ('Tirthankar Sarkar', 'Zé Rafael'),
 ('Tirthankar Sarkar', 'Václav Klán'),
 ('Tirthankar Sarkar', 'Élder Santana'),
 ('Tirthankar Sarkar', 'Welinton Júnior'),
 ('Tirthankar Sarkar', 'Ângelo Meneses'),
 ('Tirthankar Sarkar', 'Wendell (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Wilson Kenidy'),
 ('Tirthankar Sarkar', 'Tomás Paschetta'),
 ('Tirthankar Sarkar', 'Tomás Spinelli'),
 ('Tirthankar Sarkar', 'Yohan Boli'),
 ('Tirthankar Sarkar', 'Vitinho (footballer, born October 1993)'),
 ('Tirthankar Sarkar', 'Vitinho (footballer, born October 1993)'),
 ('Tirthankar Sarkar', 'Víctor Ruiz (footballer, born 1993)'),
 ('Wut Tola', 'Álvaro Carrillo (footballer)'),
 ('Wut Tola', 'Álex Blesa'),
 ('Wut Tola', 'Óscar Aranda'),
 ('Chhom Pisa', 'Nub Tola'),

### Regra 2

Acha todos os jogadores parceiros, ou seja, que jogam no mesmo clube

In [14]:
def jogadores_parceiros_consulta_aberta(df):
    return [(j1, j2) for j1, clube1 in zip(df['nome'], df['clubeNome'])
                     for j2, clube2 in zip(df['nome'], df['clubeNome'])
                     if j1 < j2 and clube1 == clube2]

In [15]:
def jogadores_parceiros_consulta_fechada(df, nomeJogador):
    return list(filter(lambda j: j != nomeJogador and df[df['nome'] == nomeJogador]['clubeNome'].values[0] == df[df['nome'] == j]['clubeNome'].values[0], df['nome']))

In [16]:
jogadores_parceiros_consulta_aberta(new_df)

[('Chhom Pisa', 'Lim Pisoth'),
 ('Chhom Pisa', 'In Sodavid'),
 ('Chhom Pisa', 'Valeriy Hryshyn'),
 ('Sa Ty', 'Wut Tola'),
 ('Sa Ty', 'Thierry Bin'),
 ('Reung Bunheing', 'Wut Tola'),
 ('Reung Bunheing', 'Sa Ty'),
 ('Reung Bunheing', 'Thierry Bin'),
 ('Chrerng Polroth', 'Wut Tola'),
 ('Chrerng Polroth', 'Sa Ty'),
 ('Chrerng Polroth', 'Reung Bunheing'),
 ('Chrerng Polroth', 'Leng Nora'),
 ('Chrerng Polroth', 'Keo Soksela'),
 ('Chrerng Polroth', 'Paulo Victor (footballer, born 1994)'),
 ('Chrerng Polroth', 'Marcos Vinícius (footballer, born 1994)'),
 ('Chrerng Polroth', 'Thierry Bin'),
 ('Chrerng Polroth', 'Pak Song-chol (footballer, born 1987)'),
 ('Chrerng Polroth', 'Lee Jae-gun'),
 ('Chrerng Polroth', 'Kenny Athiu'),
 ('Leng Nora', 'Wut Tola'),
 ('Leng Nora', 'Sa Ty'),
 ('Leng Nora', 'Reung Bunheing'),
 ('Leng Nora', 'Paulo Victor (footballer, born 1994)'),
 ('Leng Nora', 'Marcos Vinícius (footballer, born 1994)'),
 ('Leng Nora', 'Thierry Bin'),
 ('Leng Nora', 'Pak Song-chol (footballer

In [17]:
jogadores_parceiros_consulta_fechada(new_df, "Lucas Piazon")

['Hugo (footballer, born 2001)',
 'Hugo (footballer, born 2001)',
 'Adryelson',
 'Adryelson',
 'Lucas Mezenga',
 'Lucas Mezenga',
 'Kanu (footballer, born 1997)',
 'Kanu (footballer, born 1997)',
 'Matheus Nascimento (footballer, born 2004)',
 'Matheus Nascimento (footballer, born 2004)',
 'Victor Sá',
 'Victor Sá',
 'Fernando Marçal',
 'Fernando Marçal',
 'Philipe Sampaio',
 'Philipe Sampaio',
 'Carlinhos (footballer, born November 1986)',
 'Carlinhos (footballer, born November 1986)',
 'Joel Carli',
 'Joel Carli',
 'Breno (footballer, born 2000)',
 'Breno (footballer, born 2000)',
 'Patrick de Paula',
 'Patrick de Paula',
 'Jeffinho',
 'Jeffinho',
 'Júnior Santos (footballer, born 1994)',
 'Júnior Santos (footballer, born 1994)',
 'Daniel Cruz (footballer, born 2001)',
 'Daniel Cruz (footballer, born 2001)',
 'Romildo Del Piage',
 'Romildo Del Piage',
 'Tiquinho Soares',
 'Tiquinho Soares',
 'Gatito Fernández',
 'Gatito Fernández',
 'Renzo Saravia',
 'Renzo Saravia',
 'Rafael (footba

### Regra 3

Acha todos os jogadores que são super parceiros, ou seja, jogam no mesmo clube, nasceram no mesmo pais e jogam em posições diferentes

In [18]:
def jogadores_super_parceiros(df):
    return [(j1, j2, pais) for j1, clube1, pais, pos1 in zip(df['nome'], df['clubeNome'], df['paisNascimento'], df['posicaoLabel'])
                            for j2, clube2, pais2, pos2 in zip(df['nome'], df['clubeNome'], df['paisNascimento'], df['posicaoLabel'])
                            if j1 < j2 and clube1 == clube2 and pais == pais2 and pos1 != pos2]

In [19]:
jogadores_super_parceiros(new_df)

[('Chhom Pisa', 'Lim Pisoth', 'Cambodia'),
 ('Chhom Pisa', 'In Sodavid', 'Cambodia'),
 ('Chrerng Polroth', 'Wut Tola', 'Cambodia'),
 ('Chrerng Polroth', 'Sa Ty', 'Cambodia'),
 ('Chrerng Polroth', 'Reung Bunheing', 'Cambodia'),
 ('Chrerng Polroth', 'Leng Nora', 'Cambodia'),
 ('Chrerng Polroth', 'Keo Soksela', 'Cambodia'),
 ('Leng Nora', 'Wut Tola', 'Cambodia'),
 ('Leng Nora', 'Sa Ty', 'Cambodia'),
 ('Leng Nora', 'Reung Bunheing', 'Cambodia'),
 ('Nub Tola', 'Sou Yaty', 'Cambodia'),
 ('Keo Soksela', 'Wut Tola', 'Cambodia'),
 ('Keo Soksela', 'Sa Ty', 'Cambodia'),
 ('Keo Soksela', 'Reung Bunheing', 'Cambodia'),
 ('Keo Soksela', 'Leng Nora', 'Cambodia'),
 ('In Sodavid', 'Lim Pisoth', 'Cambodia'),
 ('Nen Sothearoth', 'Sou Yaty', 'Cambodia'),
 ('Marcos Vinícius (footballer, born 1994)',
  'Paulo Victor (footballer, born 1994)',
  'Brazil'),
 ('Hugo (footballer, born 2001)',
  'Matheus Nascimento (footballer, born 2004)',
  'Brazil'),
 ('Hugo (footballer, born 2001)',
  'Matheus Nascimento (foo

### Regra 4

Acha todos os centro avantes goleadores, ou seja, jogadores camisa 9, na posição de atacante que tenham feito mais de 50 gols

Nesta regra foi utilizada a funcionalidade **```reduce```**

In [20]:
def centro_avante_goleador_consulta_aberta(df):
    return df[(df['posicaoLabel'] == "Forward (association football)") &
              (df['numeroCamisa'] == 9) &
              (df['allGols'] > 50)]

In [21]:
def centro_avante_goleador_consulta_fechada(df, nomeJogador):
    return reduce(lambda acc, j: acc or (j['nome'] == nomeJogador and j['posicaoLabel'] == "Forward (association football)" 
                                         and j['numeroCamisa'] == 9 and j['allGols'] > 50), df.to_dict(orient='records'), False)

In [22]:
centro_avante_goleador_consulta_aberta(new_df)

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
544,Elkeson,Brazil,1989-07-13,Grêmio Foot-Ball Porto Alegrense,Forward (association football),1.80,9,59,Campeonato Brasileiro Série A
1335,Mersudin Ahmetović,Bosnia & Herzegovina,1985-03-19,FK Sarajevo,Forward (association football),1.91,9,54,Premier League of Bosnia and Herzegovina
1587,Nahikari García,Spain,1997-03-10,Real Madrid Femenino,Forward (association football),1.59,9,104,Primera División (women)
2213,Jeison Medina,Colombia,1995-02-27,Al-Shamal SC,Forward (association football),1.80,9,65,Qatar Stars League
2360,Abdulfattah Adam,Saudi Arabia,1995-01-01,Abha Club,Forward (association football),1.80,9,51,Saudi Professional League


In [23]:
centro_avante_goleador_consulta_fechada(new_df, "Jeison Medina")

True

### Regra 5

Acha todos os jogadores concorrentes, ou seja, que jogam no mesmo clube e jogam na mesma posição

In [24]:
def jogadores_concorrentes(df):
    return [(j1, j2, pos, clube) for j1, pos, clube in zip(df['nome'], df['posicaoLabel'], df['clubeNome'])
                                    for j2, pos2, clube2 in zip(df['nome'], df['posicaoLabel'], df['clubeNome'])
                                    if j1 < j2 and pos == pos2 and clube == clube2]

In [25]:
jogadores_concorrentes(new_df)

[('Sa Ty', 'Wut Tola', 'Forward (association football)', 'Visakha FC'),
 ('Reung Bunheing',
  'Wut Tola',
  'Forward (association football)',
  'Visakha FC'),
 ('Reung Bunheing', 'Sa Ty', 'Forward (association football)', 'Visakha FC'),
 ('Chrerng Polroth',
  'Thierry Bin',
  'Midfielder (association football)',
  'Visakha FC'),
 ('Nen Sothearoth',
  'Nub Tola',
  'Defender (association football)',
  'Nagaworld FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Wut Tola',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Sa Ty',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Reung Bunheing',
  'Forward (association football)',
  'Visakha FC'),
 ('Seon Power',
  'Sor Rotana',
  'Defender (association football)',
  'Prey Veng FC'),
 ('Takashi Odawara',
  'Víctor Blasco',
  'Midfielder',
  'Preah Khan Reach Svay Rieng FC'),
 ('Lee Jae-gun', 'Wut Tola', 'Forward (association football)', 'Visa

### Regra 6

Acha todos os jogadores rivais, ou seja, que jogam na mesma liga e em clubes diferentes

In [26]:
def jogadores_rivais(df):
    parceiros = jogadores_parceiros_consulta_aberta(df)
    return [(j1, j2, liga) for j1, clube1, liga in zip(df['nome'], df['clubeNome'], df['liga'])
                              for j2, clube2, liga2 in zip(df['nome'], df['clubeNome'], df['liga'])
                              if j1 < j2 and liga == liga2 and clube1 != clube2
                              and (j1, j2) not in parceiros and (j2, j1) not in parceiros]

In [27]:
jogadores_rivais(new_df)

[('Chhom Pisa', 'Wut Tola', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Sa Ty', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Reung Bunheing', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Chrerng Polroth', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Leng Nora', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Nub Tola', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Keo Soksela', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Phuong Soksana', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Sou Yaty', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Nen Sothearoth', 'Cambodian Premier League'),
 ('Chhom Pisa', 'Maksim Taleyko', 'Cambodian Premier League'),
 ('Chhom Pisa',
  'Paulo Victor (footballer, born 1994)',
  'Cambodian Premier League'),
 ('Chhom Pisa',
  'Marcos Vinícius (footballer, born 1994)',
  'Cambodian Premier League'),
 ('Chhom Pisa', 'Thierry Bin', 'Cambodian Premier League'),
 ('Chhom Pisa',
  'Pak Song-chol (footballer, born 1987)',
  'Cambodian Premier League'),
 ('Chhom P

### Regra 7

Acha todos os goleiros bons, ou seja, jogadores que jogam na posição de goleiro, que tenham pelo menos 1.90 de altura e que tenham feito pelo menos 1 gol

In [28]:
def goleiro_bom(df):
    return df[(df['posicaoLabel'] == "Goalkeeper (association football)") &
              (df['altura'] >= 1.90) &
              (df['allGols'] >= 1)]

In [29]:
goleiro_bom(new_df)

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
546,"Vinícius (footballer, born 1984)",Brazil,1984-11-09,Clube do Remo,Goalkeeper (association football),1.90,1,1,Campeonato Brasileiro Série C
583,Pegorari,Brazil,1991-07-12,Esporte Clube Juventude,Goalkeeper (association football),1.91,1,1,Campeonato Brasileiro Série A
594,Eduardo Martini,Brazil,1979-01-11,Grêmio Esportivo Brasil,Goalkeeper (association football),1.92,1,1,Campeonato Brasileiro Série D
706,"Everson (footballer, born 1990)",Brazil,1990-07-22,Clube Atlético Mineiro,Goalkeeper (association football),1.92,22,1,Campeonato Brasileiro Série A
707,"Everson (footballer, born 1990)",Brazil,1990-07-22,Clube Atlético Mineiro,Goalkeeper (association football),1.92,22,1,Campeonato Mineiro
1394,"Paulo Vítor (footballer, born 1988)",Brazil,1988-11-21,G.D. Chaves,Goalkeeper (association football),1.90,28,1,Primeira Liga
2576,Vladimir Stojković,Serbia,1983-07-28,Al-Fayha FC,Goalkeeper (association football),1.95,88,1,Saudi Professional League


### Regra 8

Acha todos os jogadores camisa 10 da shoppe, ou seja, jogadores que tem o número de camisa 10, e que não tenham feito nenhum gol

In [30]:
def jogadores_camisa_10_da_shoppe(df):
    return df[(df['numeroCamisa'] == 10) & (df['allGols'] == 0)]

In [31]:
jogadores_camisa_10_da_shoppe(new_df)

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
1191,Michael Modubi,South Africa,1985-04-22,K.F.C. Dessel Sport,Midfielder,1.76,10,0,Belgian National Division 1
2065,Ismael Gutiérrez,Spain,2000-10-10,CF La Nucía,Midfielder,1.76,10,0,Primera División RFEF
2297,Jameel Al-Yahmadi,Oman,1996-07-27,Al-Markhiya SC,Forward (association football),1.68,10,0,Qatar Stars League
2298,Jameel Al-Yahmadi,Oman,1996-07-27,Al-Markhiya SC,Midfielder (association football),1.68,10,0,Qatar Stars League
2300,Salaah Al-Yahyaei,Oman,1998-08-17,Qatar SC,Midfielder (association football),1.80,10,0,Qatar Stars League
2340,Thaar Al-Otaibi,Saudi Arabia,1999-08-14,Al-Hazem F.C.,Midfielder,1.73,10,0,Saudi First Division League
